<a href="https://colab.research.google.com/github/rochan-saireddy/influenza-protein-cleavage-enzyme/blob/main/PDB_CLEANUP_BIOPY.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install biopython
from Bio import PDB
import os

# Paths
input_files = ['1sbt.pdb', '3wqb.pdb']

# Set up output folders
os.makedirs('/content/clean', exist_ok=True)
os.makedirs('/content/backbone', exist_ok=True)

# Cleaning and backbone strip
for pdb_file in input_files:
    parser = PDB.PDBParser(QUIET=True)
    structure = parser.get_structure(pdb_file.split('.')[0], pdb_file)

    # CLEAN: remove all chains except A, and non-AA residues
    class CleanSelect(PDB.Select):
        def accept_chain(self, chain):
            return chain.get_id() == 'A'
        def accept_residue(self, residue):
            return residue.id[0] == ' '

    clean_out = f'/content/clean/{pdb_file}'
    io = PDB.PDBIO()
    io.set_structure(structure)
    io.save(clean_out, CleanSelect())

    # RELOAD cleaned structure
    structure_clean = parser.get_structure(f'{pdb_file}_clean', clean_out)

    # STRIP: keep only backbone atoms
    class BackboneOnly(PDB.Select):
        def accept_atom(self, atom):
            return atom.get_name() in ['N', 'CA', 'C', 'O']

    backbone_out = f'/content/backbone/{pdb_file.split(".")[0]}_backbone.pdb'
    io.set_structure(structure_clean)
    io.save(backbone_out, BackboneOnly())

    print(f'[✔] Cleaned: {clean_out}')
    print(f'[✔] Backbone: {backbone_out}')


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 34.9 MB/s eta 0:00:00
[✔] Cleaned: /content/clean/1sbt.pdb
[✔] Backbone: /content/backbone/1sbt_backbone.pdb
[✔] Cleaned: /content/clean/3wqb.pdb
[✔] Backbone: /content/backbone/3wqb_backbone.pdb
